In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from preprocess import GSE
from models import AutoEncoder, VariationalAutoEncoder, VariationalDeepEmbedding, PlotLatentSpace

In [6]:
dataset = GSE()
X_train, y_train = dataset.train
X_test, y_test = dataset.test

In [7]:
model = AutoEncoder(original_dim=dataset.n_genes)

In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=20)
plot_latent = PlotLatentSpace(model, X_train, y_train, interval=10)
model.compile(optimizer=Adam(0.0001), loss=model.reconstruction_loss)
history = model.fit(X_train, X_train, epochs=1000, validation_data=(X_test, X_test),
                   callbacks=[early_stopping, plot_latent], verbose=True)

Train on 322 samples, validate on 108 samples
Epoch 1/1000
320/322 [============================>.] - ETA: 0s - loss: 3954.9231WARNING:tensorflow:Layer Encoder is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

322/322 [==============================] - 10s 30ms/sample - loss: 3954.2470 - val_loss: 3655.4815
Epoch 2/1000
322/322 [==============================] - 4s 12ms

KeyboardInterrupt: 

In [13]:
from scipy.optimize import linear_sum_assignment
from sklearn.utils.linear_assignment_ import linear_assignment

/home/alumnos/secancino/anaconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [16]:
import numpy as np
a = linear_sum_assignment(np.eye(10))

In [15]:
linear_assignment(np.eye(10))

/home/alumnos/secancino/anaconda3/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


array([[0, 1],
       [1, 0],
       [2, 3],
       [3, 2],
       [4, 5],
       [5, 4],
       [6, 7],
       [7, 6],
       [8, 9],
       [9, 8]])

In [20]:
for i, j in zip(*a):
    print(i, j)

0 1
1 0
2 3
3 2
4 5
5 4
6 7
7 6
8 9
9 8
